<a href="https://colab.research.google.com/github/knodlerc/gemini-carreiras/blob/main/projeto-gemini-carreiras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#-------------------------------------------------------------------------------
# Imersão IA_ (Alura e Google)
#-------------------------------------------------------------------------------
# Projeto Gemini Carreiras
#-------------------------------------------------------------------------------
#
# Responsável: knodlerc
# Versão: 1.0
# Data: 16/Maio/2025
# License Type: GNU v3.0
#
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# Avisos importantes:
#
#   * Tenha em mãos seu currículo. Você precisará fornece-ló o agente.
#     Não compartilhe informações pessoais no seu currículo. Temos um agente
#     que tentará remover informações pessoais baseado na LGPD
#     Exemplo de informações pessoais: nome,endereço,telefone,e-mail,RG,CPF
#
#   * Se você não quiser compartilhar seu currículo, verifique no Github os
#     currículos de testes que disponibilizamos
#
#   * Você precisará de uma chave de API do Google Gemini
#     Documentação: https://ai.google.dev/gemini-api/docs
#
#   * Para visualizar o resultado de cada agente envolvido no processo altere
#     a variável _DEBUG_AGENTS para True. O padrão é deixar esta opção
#     desativada (Valor da variável: False)
#
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# Objetivo:
#   O projeto "Gemini Carreiras" visa desenvolver um agente de Inteligência
#   Artificial sofisticado, construído sobre a arquitetura do Google Gemini,
#   capaz de atuar como um consultor de carreira personalizado. A ferramenta
#   analisa currículos enviados por usuários e, com base em uma profunda
#   compreensão do mercado de trabalho atual, oferece insights valiosos e
#   recomendações práticas para o aprimoramento de habilidades técnicas e
#   comportamentais. O objetivo principal é capacitar os indivíduos a alinhar
#   seus perfis profissionais com as demandas do mercado, aumentando suas
#   chances de sucesso em suas jornadas de carreira.
#
# Fluxo do Gemini Carreiras
#   a) Dar boas vindas e informar quais informações serão solicitadas
#   b) Identificar a profissão da pessoa
#   c) Identificar qual vaga a pessoa está buscando
#   d) Solicitar o currículo
#   e) Verificar a nessidade de tradução do currículo para Portugues Brasil
#   f) Devido a Lei Geral de Proteção de Dados (LGPD) analisar CV e remover
#      informações pessoais
#   g) Buscas no google vagas de empregos de acordo com a profissão em sites
#      pré-definidos e identificar as qualidades e habilidades solicitadas
#   h) Analisar e comparar o curriculo enviado com as informações solicitadas
#      pelo mercado
#   i) Revisar e apresentar o resultado
#
#
# Sites de vagas de empregos utilizado:
#     https://www.infojobs.com.br/
#     https://www.catho.com.br/vagas/
#     https://br.indeed.com/
#     https://www.jobatus.com.br/
#     https://www.vagas.com.br/
#     https://portal.gupy.io/
#     https://riovagas.com.br/
#     https://99jobs.com/
#
# Histórico:
#   Data: 16/Maio/2025
#   Detalhes: Versão inicial
#
#-------------------------------------------------------------------------------

################################################################################
# Instalando bibliotecas necessárias
!pip install -qU google-genai
!pip install -qU google-adk

################################################################################
# Importando bibliotecas
import os
import time
import io
import textwrap
import requests
import warnings
from IPython.display import HTML, Markdown
from IPython.display import display
from google.colab import userdata
from google import genai
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
from google.colab import output
from google.colab import files


warnings.filterwarnings("ignore")

################################################################################
# Definindo os parâmetros

# Obtendo chave API do Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Modelo to Gemini
_MODELO_GENAI = "gemini-2.0-flash"

# Utilizado apenas no último agenda para disponsibilizar informações detalhes
# sobre a análise do currículo - Deixou de funcionar: gemini-2.5-flash-preview-04-17-thinking
_MODELO_GENAI_THINKING = "gemini-2.0-flash"

# Prompt timer
_PROMPT_TIMER = 1.5

# Relação de sites de busca de vagas
_STIE_BUSCA_VAGAS = "https://www.infojobs.com.br , https://www.catho.com.br/vagas , https://br.indeed.com , https://www.jobatus.com.br , https://www.vagas.com.br , https://portal.gupy.io , https://riovagas.com.br , https://99jobs.com, https://www.roberthalf.com/br/pt/vagas"

# Debug dos Agentes
#   Mostra o resultado de cada agente durante o processo de avaliação
#     True - a mensagem será mostrada
#     False - a mensage será omitida (padrão para melhor a experiência do usuário)
_DEBUG_AGENTS = False

# Variável orientações
#_strOrientacoes = "não" # valor utilizado nos testes
_strOrientacoes = "n"

# Variável profissão
_strProfissao = ""

# Variável vagas
_strVaga = ""

# Variável currículo
_strCurriculo = ""

#Gemini API client
_objClient = genai.Client()

#Data Hoje
_dthDataAtual = date.today().strftime("%d/%m/%Y")

#for _objModel in _objClient.models.list():
#  print(_objModel.name)

################################################################################
# Funções
################################################################################

################################################################################
# Nome: _print
# Objetivo: otimizar o código fonte e garantir padrão na saída de infomrçãoes
# Parâmetros de entrada:
#   p_strTexto - Texto a ser apresentado no Notebook
#   p_fltSegundos - segundos de espera após o prompt
#   p_blnTratarText - define se o texto deve ser tratado
# Parâmetro de saída:
#   Sem retorno
def _print(p_strTexto, p_fltSegundos=0, p_blnTratarTexto=False):
  if p_blnTratarTexto:
    p_strTexto = p_strTexto.replace('•', '  *')
    p_strTexto =textwrap.indent(p_strTexto.replace('•', '  *'), '> ', predicate=lambda _: True)

  display(Markdown(f"{p_strTexto}"))

  if p_fltSegundos > 0:
    time.sleep(p_fltSegundos)

################################################################################
# Nome: _comentarios_profissao
# Objetivo: Fazer um comentário sobre a profissão
# Parâmetros de entrada:
#   p_strProfissao - Profissão informada no prompt
# Parâmetro de saída:
#   retorno - Texto gerado pelo Gemini
def _comentarios_profissao(p_strProfissao):
  objClient = genai.Client()

  objChat_Config = types.GenerateContentConfig(
    system_instruction="Você é um assistente pessoal e sempre responde de forma suscita e positiva." )

  objChat = objClient.chats.create(model=_MODELO_GENAI,config=objChat_Config)
  return objChat.send_message("Fazer um comentário positivo sobre a profissão " + p_strProfissao).text

################################################################################
# Nome: _avancos_profissao
# Objetivo: Fazer um comentário sobre os vanços da profissão
# Parâmetros de entrada:
#   p_strProfissao - Profissão informada no prompt
# Parâmetro de saída:
#   retorno - Texto gerado pelo Gemini
def _avancos_profissao(p_strProfissao):
  objClient = genai.Client()

  objChat_Config = types.GenerateContentConfig(
    system_instruction="""Você é um assistente pessoal com vasto conhecimento técnico em todas as profissões.
                          Cada item listado deve ter ao menos duas linhas de informações.""" )

  objChat = objClient.chats.create(model=_MODELO_GENAI,config=objChat_Config)
  return objChat.send_message(f"Fazer um relação de cinco avanços tecnologócios impactantes referente a profissão {p_strProfissao}").text

################################################################################
# Nome: _chamar_agenteIA
# Objetivo: Chamada de um agente IA com tarefa a ser definida
# Parâmetros de entrada:
#   p_objAgente - objeto orientações agente
#   p_strMensagem - segundos de espera após o prompt
# Parâmetro de saída:
#   Resposta agente
def _chamar_agenteIA(p_objAgente: Agent, p_strMensagem: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=p_objAgente.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=p_objAgente, app_name=p_objAgente.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=p_strMensagem)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


################################################################################
# Nome: _agente_tradutor
# Objetivo: Agente de IA que irá traduzir o CV se necessário
# Parâmetros de entrada:
#   strTexto - Texto a ser apresentado no Notebook
# Parâmetro de saída:
#   retorno - Texto gerado pelo Gemini
def _agente_tradutor(p_strTexto, p_strIdiomaDestino="Português do Brasil"):

    objAgente = Agent(
        name="agente_tradutor",
        model=_MODELO_GENAI,
        instruction="""
        Você é um tradutor de idiomas com conhecimento em todos os idiomas existentes.
        A sua tarefa é traduzir o texto informado para o idioma de destino desejado.
        Identifique o contexto do texto informado e realize a tradução da melhor forma possível.
        Você não deve adicionar nenhum comentário ao texto.

        """,
        description="Agente que realiza a tradução para o idioma de destino",
        tools=[google_search]
    )

    strMensagem = f"Texto informado: {p_strTexto}\nIdioma de destino desejado: {p_strIdiomaDestino}"

    strTextoTraduzido = _chamar_agenteIA(objAgente, strMensagem)

    #Apresentar resultado do agente
    if _DEBUG_AGENTS:
      _print("Resultado do agente de tradução:")
      _print("<br>")
      _print(p_strTexto=strTextoTraduzido, p_blnTratarTexto=True)
      input("Pressione ENTER >> ")
      output.clear()
    return strTextoTraduzido

################################################################################
# Nome: _agente_lgpd
# Objetivo: Agente de IA que irá atuar na proteção de dados pessoais e remover
#   informções pessoais
# Parâmetros de entrada:
#   strTexto - Texto a ser apresentado no Notebook
# Parâmetro de saída:
#   retorno - Texto gerado pelo Gemini
def _agente_lgpd(p_strCurriculo):

    objAgente = Agent(
        name="agente_lgpd",
        model=_MODELO_GENAI,
        instruction="""
        Você é um oficial de proteção de dados com profundo conhecimento na lei geral de proteção de dados(LGPD) aplicada no Brasil.
        A sua tarefa é analisar o currículo apresentados abaixo e remover informções pessoais como nome, sobrenome, idade,
        data de nascimento, estado civil, número do registro geral (RG), número do cadastro de pessoa física (CPF),
        número do passaporte, e-mail, endereço entre outras informações pessoais.
        Como você está analisando currículos, as informações como gradução, pós-graduação, doutorado, mestrado, cursos
        de especializações em escolas e universidades são permitidos e não devem ser removiodos.
        Experiência profissifional, habilidades, idiomas, competências, formação acadêmica, cursos, treinamentos, ferramentas e conhecmentos não devem ser removidas.
        Você deve apenas remover as informações pessoais definidas acima sem alterar as informações originais .
        Você deverá retornar todoas as informações do currículo removendo apenas as informações pessoais conforme definido acima. Não adicione outros comentários ao texto.
        """,
        description="Agente que realiza a analise do currículo e garante a proteção de dados pessoais",
        tools=[google_search]
    )

    strMensagem = f"Currículo: {p_strCurriculo}\n"

    strCurriculoLGPD = _chamar_agenteIA(objAgente, strMensagem)

    #Apresentar resultado do agente
    if _DEBUG_AGENTS:
      _print("Resultado do agente de proteção de dados pessoais - LGPD:")
      _print("<br>")
      _print(p_strTexto=strCurriculoLGPD, p_blnTratarTexto=True)
      input("Pressione ENTER >> ")
      output.clear()

    return strCurriculoLGPD

################################################################################
# Nome: _agente_buscador_vagas
# Objetivo: Agente de IA que irá buscar informações de vagas
# Parâmetros de entrada:
#   p_strVaga - vaga no qual estamos buscando informações
#   p_dthData - Data atual
# Parâmetro de saída:
#   retorno - detalhes das vagas
def _agente_buscador_vagas(p_strVaga, p_dthData):

    objAgente = Agent(
        name="agente_buscador_vagas",
        model=_MODELO_GENAI,
        instruction="""
        Você é um especialista da área de recursos humanos com vasto conhecimento na busca de vagas de empregos e relacionar as informações solicitadas pelas vagas.
        A sua tarefa é usar a ferramenta de busca do google (google_search) para pesquisar informações da vagas  utilizando a relação de sites de vagas pré-definidos.
        Você deverá realizar a busca de ao menos 50 vagas relacionadas a vaga desejada que estamos buscando e extrair  uma listar detalhada das seguintes informações desejadas:
        experiencia desejada, formação acadêmica, idiomas, habilidades técnicas e comportamentais, cursos, ferramentas e conhecimentos.
        Sua lista deverá ser crtiada com base em tododas as vagas buscadas e informações coletadas. Você devé listar as informações nos tópicos:
        experiencia desejada, formação acadêmica, idiomas, habilidades técnicas e comportamentais, cursos, ferramentas e conhecimentos
        As vagas devem ser atuais, de no máximo seis meses antes da data de hoje.
        Não adicione outros comentários ao texto somente relacione as informações solicitadas.
        """,
        description="Agente que busca informações de vagas no qual o canditado está procurando",
        tools=[google_search]
    )

    strMensagem = f"Vaga desejada: {p_strVaga}\nSites de vagas pré-definidos: {_STIE_BUSCA_VAGAS}\nData de hoje: {p_dthData}\n"

    strInformacoesVagas = _chamar_agenteIA(objAgente, strMensagem)
    #Apresentar resultado do agente
    if _DEBUG_AGENTS:
      _print("Resultado do agente buscador de vagas:")
      _print("<br>")
      _print(p_strTexto=strInformacoesVagas, p_blnTratarTexto=True)
      input("Pressione ENTER >> ")
      output.clear()

    return strInformacoesVagas


################################################################################
# Nome: _agente_buscador_cursos
# Objetivo: Agente de IA que irá buscar cursos
# Parâmetros de entrada:
#   p_strProfissao - Profissão informada
#   p_strVaga - Vaga desejada
#   p_strInformacoesVagas - Informações coletadas das vagas
# Parâmetro de saída:
#   retorno - detalhes dos cursos
def _agente_buscador_cursos(p_strProfissao, p_strVaga, p_strInformacoesVagas):

    objAgente = Agent(
        name="agente_buscador_cursos",
        model=_MODELO_GENAI,
        instruction="""
        Você é um especialista em treinamento com muitos anos de experiencia buscando e ofertando cursos
        e universiários para pessosas. Utilize a ferramenta de busca do google (google_search) para
        pesquisar cursos ou universiários com relação a profissão informada, a vaga informada e as informações das vagas.
        Verifique se a profissão informada está relacionada a vaga que a pessoa está buscando. Se for diferente, pesquise
        cursos apenas baseado na vaga informada.
        Se a profissão for da área de tecnologia da informação, concentre-se primeiro em cursos
        da Alura em http://www.alura.com.br. Se for outra profissão utilize a ferramenta de busca citada acima
        para pesquisar opções. Para cursos universiários, pequise primeiro cursos de graduação, pos-graduação e
        MBA da FIAP em https://www.fiap.com.br. Caso não encontre correlação com os cursos ofertados e o relação
        profissão informada e as informações das vagas, pesquise cursos universitários nas principais
        universidades do Brasil.
        Você deve compiliar uma lista separando os cursos e cursos universitário. As informações apresentadas
        devem conter detalhes sobre o nome do curso, qual instituilção oferece este curso e os detalhes do curso.
        A lista deve conter no mínimo 5 cursos e 5 cursos universitários.
        """,
        description="Agente que busca informações de cursos relacionados a profissão e informações das vagas",
        tools=[google_search]
    )

    strMensagem = f"Profissão informada: {p_strProfissao}\nVaga informada: {p_strVaga}\nInformações das vagas: {p_strInformacoesVagas}\n"

    strCursos = _chamar_agenteIA(objAgente, strMensagem)
    #Apresentar resultado do agente
    if _DEBUG_AGENTS:
      _print("Resultado do agente buscador de cursos:")
      _print("<br>")
      _print(p_strTexto=strCursos, p_blnTratarTexto=True)
      input("Pressione ENTER >> ")
      output.clear()

    return strCursos


################################################################################
# Nome: _agente_recursos_humanos
# Objetivo: Agente de IA que irá analisar o currículo e comprar com o que está
#   sendo solicitado ao mercado. Também irá propor conhecimentos que a pessoa
#   deve adquirirs
# Parâmetros de entrada:
#   p_strVaga - vaga no qual estamos buscando informações
#   p_strCurriculoLGPD - currículo após LGPD
#   p_strInformacoesVagas - Informações das vagas
# Parâmetro de saída:
#   retorno - análise
def _agente_recursos_humanos(p_strProfissao, p_strVaga, p_strCurriculoLGPD, p_strInformacoesVagas):

    strCursosPesquidados = _agente_buscador_cursos(p_strProfissao, p_strVaga, p_strInformacoesVagas)

    objAgente = Agent(
        name="agente_recursos_humanos",
        model=_MODELO_GENAI_THINKING,
        instruction="""
        Você é um consultor de carreira e um especialista da área de recursos humanos com vasto conhecimento em contratação de
        profissionais para a vaga desejada.
        A sua tarefa é comparar o currículo fornecido com as informações das vagas pesquisadas. Tente identificar pontos em comum entre
        o currículo e as informações das vagas. Identifique os pontos positivos que o canditado possui com relação das informações das
        vagas pesquisadas. Identifique também os pontos de melhorias que o candidato deve se concentrar e buscar aperfeiçoamento.
        Utilize os cursos pesquisados para propor estas melhorias.
        Também verifique habilidades comportamentais e inclua na suia lista.
        Seja sempre positivo mas realista em seus comentários. É importante a pessoa entender os pontos que precisa melhorar.
        Você também deve incentivar a busca de novos conhecimentos.
        """,
        description="Agente que analisa o curriculo e as informações das vagas e compila uma proposta",
        tools=[google_search]
    )

    strMensagem = f"Vaga desejada: {p_strVaga}\nCurrículo fornecido: {p_strCurriculoLGPD}\nInformações das vagas pesquisadas: {p_strInformacoesVagas}\nCursos pesquisados: {strCursosPesquidados}"

    strAnaliseRH = _chamar_agenteIA(objAgente, strMensagem)

    return strAnaliseRH


################################################################################
#Início do programa
################################################################################


output.clear()

#Boas vindas
_print("****")
_print("#**Imersão IA_ | Alura e Google | Gemini Carreiras**")
_print("****")
_print("<br>")
_print("🤖 Seja bem-vindo!", _PROMPT_TIMER)

# Avisos importantes
_print("<br>")
_print("🤖 Gostaria de receber algumas orientações sobre a proteção de dados pessoais antes de começarmos.", _PROMPT_TIMER)

# Obteando a orientações
if not _strOrientacoes:
  while _strOrientacoes == "":
    _strOrientacoes = input("🧑 (sim/não) >> ").strip().lower()
    if _strOrientacoes not in {"sim", "s", "nao", "n", "yes", "y", "no"}:
      _print("<br>")
      _print("🤖 Infelizmente não consegui entender sua resposta. Vou mostrar as orientações. 😊", _PROMPT_TIMER)
      _strOrientacoes = "s"

if _strOrientacoes == "sim" or _strOrientacoes == "s":
  _print("###ATENÇÃO: Para sua segurança nunca compartilhe informações pessoais ou confidenciais da sua empresa como agentes de IA.")
  _print("<br>")
  _print("Exemplo de informações pessoais:")
  _print("* Nome, endereço, data de nascimento")
  _print("* Documentos como: RG, CPF, Passaporte")
  _print("<br>")
  _print("Exemplo de informações confidenciais das empresas:")
  _print("* Dados financeiros como balanço, planilhas eletrônicas com despesas, dados bancários")
  _print("* Desenhos industriais e código fonte")
  _print("* Arquivos com informações de funcionários")

################################################################################
# Informações necessárias
_print("<br>")
_print("🤖 Agora precisaremos de algumas informações suas como: profissão, a vaga que está buscando e seu currículo.", _PROMPT_TIMER)

# Obteando a profissão
_print("<br>")
_print("🤖 Vamos começar. Qual a sua profissão❓", _PROMPT_TIMER)

if not _strProfissao:
  while _strProfissao == "":
    _strProfissao = input("🧑 (Profissão) >> ").strip()
    if not _strProfissao.strip():
      _print("<br>")
      _print("🤖 Você não forneceu sua profissão. Vamos tentar novamente❗", _PROMPT_TIMER)

# Garantindo que a profissão está em Português do Brasil
_strProfissao = _agente_tradutor(_strProfissao)

# Buscando comentários da profissão
_print("<br>")
_print("🤖 🎉 " + _comentarios_profissao(_strProfissao), _PROMPT_TIMER)

# Obteando a vaga
_print("<br>")
_print("🤖 Qual a vaga que você está buscando❓ Seja bem específico.", _PROMPT_TIMER)

if not _strVaga:
  while _strVaga == "":
    _strVaga = input("🧑 (Vaga) >> ").strip()
    if not _strVaga.strip():
      _print("<br>")
      _print("🤖 Você não forneceu a vaga que está buscando. Vamos tentar novamente❗", _PROMPT_TIMER)

# Garantindo que a vaga está em Português do Brasil
_strVaga = _agente_tradutor(_strVaga)

#Obtendo o currículo em texto
_print("<br>")
_print("🤖 Está é a última informação. Copie e cole o texto do seu currículo", _PROMPT_TIMER)

while _strCurriculo == "":
  _strCurriculo = input("🧑 (Currículo) >> ").strip()
  if not _strCurriculo.strip():
    _print("<br>")
    _print("🤖 Você não forneceu seu currículo. Vamos tentar novamente❗", _PROMPT_TIMER)

# Chamando agente tradutor
_print("<br>")
_print("🤖 Verificando a necessidade de tradução do seu currículo. (👨‍🏫Agente Tradutor)", _PROMPT_TIMER)
strCurriculoTraduzido = _agente_tradutor(p_strTexto=_strCurriculo)

# Chamando agente LGPD
_print("<br>")
_print("🤖 Iremos procurar e remover dados pessoais do seu currículo. (🔒Agente de Proteção de Dados pessoais - LGPD) ", _PROMPT_TIMER)
strCurriculoAposLGPD = _agente_lgpd(strCurriculoTraduzido)

#Buscando avanços da profissão
_print("<br>")
_print("🤖 Enquanto realizamos a busca, veja uma relação de avanços na sua profissão.", _PROMPT_TIMER)
_print( _avancos_profissao(_strProfissao), _PROMPT_TIMER, True)

#Buscando vagas de emprego
_print("<br>")
_print("🤖 Agora iremos pesquisas vagas de empregos em diversos sites. (🕵️‍♀️ Agente buscador de vagas)", _PROMPT_TIMER)

strInformacoesVagas = _agente_buscador_vagas(_strVaga, _dthDataAtual)

#Analisar Currículo e comparar com o solicitado pelo mercado
_print("<br>")
_print("🤖 Agora que encontramos as informações das vagas, iremos iniciar a análise do seu currículo. (👩‍💻Agente de RH)", _PROMPT_TIMER)
strAnaliseRH = _agente_recursos_humanos(_strProfissao, _strVaga, strCurriculoAposLGPD, strInformacoesVagas)

output.clear()
_print("****")
_print("#**Imersão IA_ | Alura e Google | Gemini Carreiras**")
_print("****")
_print("<br>")
_print("##Resultado da análise realizada pelo Gemini Carreiras##")
_print("<br>")
_print(strAnaliseRH)


****

#**Imersão IA_ | Alura e Google | Gemini Carreiras**

****

<br>

##Resultado da análise realizada pelo Gemini Carreiras##

<br>

Olá! É um prazer atuar como seu consultor de carreira nesta análise. A transição de um papel técnico para uma posição de gestão é um movimento natural e ambicioso, e seu currículo demonstra uma base sólida para essa jornada. Vamos analisar seu perfil em comparação com os requisitos típicos para um Gerente de TI, identificando seus pontos fortes atuais e as áreas onde um desenvolvimento focado será mais impactante.

Seu currículo apresenta um perfil técnico promissor e ousado, com um histórico acadêmico notável e experiência prática, mesmo que em um contexto voluntário. Isso já demonstra paixão, iniciativa e capacidade de aprendizado rápido.

**Pontos Fortes e de Coincidência com as Vagas de Gerente de TI:**

1.  **Base Técnica Sólida:** Seu conhecimento em C#, C++ e Python, combinado com a formação em TI e aulas de Modelagem de Dados e IA, fornece uma compreensão profunda dos aspectos técnicos que um gerente de TI supervisionará. Sua alta performance acadêmica (média 9.5, nota 10 no TCC) reforça essa base.
2.  **Experiência Prática e Orientação para Resultados:** A participação no projeto Video Game X, que recebeu um prêmio e teve 95% de avaliações positivas, demonstra sua capacidade de aplicar conhecimentos, colaborar em equipe e contribuir para um resultado de sucesso. Implementar mecânicas, programar IA e integrar assets mostra uma visão do ciclo de desenvolvimento.
3.  **Liderança e Colaboração em Potencial:** Ser Presidente do clube de estudos e Orador da turma indica habilidades de comunicação, organização e a capacidade de influenciar e motivar grupos, características essenciais para a liderança. A colaboração com designers e artistas no projeto do jogo também evidencia uma habilidade interpessoal importante.
4.  **Habilidades Comportamentais Relevantes:** O currículo destaca explicitamente "Trabalho em equipe", "Gamificação" (que mostra uma abordagem inovadora) e implicitamente "Resolução de problemas" (necessária na programação e desenvolvimento de jogos). Sua paixão por arte e criatividade pode se traduzir em uma visão mais inovadora para soluções de TI.
5.  **Idiomas:** Seu Inglês avançado é um requisito frequente e forte para vagas de gestão, e o Espanhol intermediário é um diferencial positivo mencionado em algumas descrições.

**Áreas para Aperfeiçoamento e Desenvolvimento:**

Para alcançar a posição de Gerente de TI, a principal transição necessária é do foco técnico para o foco em *gestão*. As vagas pesquisadas enfatizam fortemente a experiência e o conhecimento em gerenciar pessoas, projetos, processos e a infraestrutura de TI de forma estratégica.

1.  **Experiência em Gestão e Liderança de Equipes:** Seu currículo demonstra potencial de liderança em contextos acadêmicos e de projeto, mas falta a experiência formal e comprovada em gerenciar equipes de TI, delegar tarefas, avaliar desempenho e desenvolver pessoas.
2.  **Gestão de Projetos (Formal):** Embora sua experiência no jogo voluntário envolva elementos de projeto, as vagas buscam vivência em metodologias formais de gerenciamento de projetos (Ágil, Scrum, Waterfall) e no uso de ferramentas de gestão (JIRA, ClickUp).
3.  **Conhecimento Abrangente de Infraestrutura e Sistemas Corporativos:** As vagas demandam conhecimento em áreas como redes, segurança da informação, servidores, backup, e sistemas específicos como ERPs (TOTVS Protheus, SAP), CRMs e ferramentas de BI. Seu foco atual é mais no desenvolvimento de software e IA.
4.  **Visão Estratégica e de Negócio:** Um Gerente de TI alinha a tecnologia aos objetivos estratégicos da empresa. É fundamental desenvolver uma visão mais ampla do negócio e de como a TI pode gerar valor, além da excelência técnica.
5.  **Habilidades Comportamentais em Contexto Gerencial:** Enquanto possui habilidades como trabalho em equipe e comunicação, é preciso desenvolvê-las para o contexto de gestão: negociação (com stakeholders, fornecedores), gestão de conflitos, tomada de decisão baseada em dados em um nível estratégico, e proatividade na identificação de oportunidades de melhoria na área de TI.

**Sugestões de Cursos para Desenvolvimento:**

Com base nas áreas de melhoria e nos cursos pesquisados, sugiro focar nas seguintes formações para complementar seu perfil técnico com as competências de gestão e infraestrutura necessárias:

1.  **Formação Gestão de Projetos de TI (Alura) ou Pós-graduação em Gerenciamento de Projetos (FIAP e outras instituições):** Essenciais para adquirir o conhecimento formal em metodologias e práticas de gestão de projetos, algo muito requisitado nas vagas.
2.  **Liderança para Gestores de TI (Alura):** Focado no desenvolvimento das habilidades interpessoais e de gestão de pessoas cruciais para liderar uma equipe de TI de forma eficaz.
3.  **Certificação ITIL 4 Foundation (Alura):** O ITIL é um framework padrão para gerenciamento de serviços de TI, abordando processos como incidentes, problemas e mudanças. Conhecê-lo é fundamental para gerenciar as operações de TI.
4.  **DevOps (Alura):** Uma prática cada vez mais importante que integra desenvolvimento e operações de TI. Conhecer DevOps demonstra uma visão moderna da entrega de valor em TI.
5.  **MBA em Gestão Estratégica de TI (FIAP e outras instituições):** Este curso oferece a visão estratégica necessária para um gerente, abordando planejamento, custos, riscos e governança, alinhando a TI aos objetivos de negócio.
6.  **MBA em Business Intelligence e Analytics (FIAP e outras instituições):** Para fortalecer a habilidade de tomar decisões baseadas em dados e utilizar ferramentas de BI, habilidades valorizadas nas vagas.
7.  **Formação em Arquitetura de Soluções (Alura) ou Pós-graduação em Arquitetura de Sistemas Distribuídos (FIAP e outras instituições):** Embora já tenha base técnica, aprofundar em arquitetura de soluções e enterprise é importante para entender a estrutura dos sistemas corporativos que gerenciará.

**Foco nas Habilidades Comportamentais:**

Além dos cursos formais, busque oportunidades para desenvolver ativamente as seguintes habilidades comportamentais no contexto de gestão:

*   **Liderança:** Procure projetos, mesmo que internos ou voluntários, onde possa assumir a responsabilidade por uma equipe ou iniciativa.
*   **Comunicação e Negociação:** Participe de workshops, pratique apresentações, e busque situações onde precise negociar escopos, prazos ou recursos.
*   **Visão Estratégica e Capacidade Analítica:** Ao analisar problemas ou oportunidades, vá além da solução técnica e pense no impacto no negócio. Utilize dados para justificar suas decisões.
*   **Organização e Gerenciamento do Tempo:** Essas habilidades são multiplicadas em importância quando você é responsável pela organização e prazos de uma equipe inteira.

Seu histórico acadêmico e sua experiência inicial mostram um grande potencial. A transição para a gestão exige um investimento consciente em novas áreas de conhecimento e no desenvolvimento de um conjunto diferente de habilidades. Sua paixão por aprender ("apaixonado por arte e criatividade", busca por aulas extracurriculares e cursos) é um grande trunfo nessa jornada. Ao focar nas áreas sugeridas e buscar ativamente oportunidades para aplicar o que aprender, você estará construindo um caminho sólido para se tornar um excelente Gerente de TI. O mercado valoriza profissionais com uma base técnica forte e que investem no desenvolvimento das competências de gestão e liderança. Siga em frente com determinação e curiosidade!
